In [ ]:
import mne
from functions.detect_bad import bad_epochs

from _parameters import *

In [ ]:
def get_epochs(s, moment, event_id, tmin, tmax, drop = False):

    # Load raw and ica
    raw_fname = dirs['raw'] + '/raw_s' + str(s) + '.fif'
    ica_fname = dirs['ica'] + '/ica_s' + str(s) + '.fif'

    raw = mne.io.read_raw_fif(raw_fname, preload = True)
    ica = mne.preprocessing.read_ica(ica_fname)

    # Apply ica
    raw = ica.apply(raw, exclude = ica.exclude)

    # Get events
    events = mne.find_events(raw, stim_channel = 'Status')
    events = mne.pick_events(events, include = triggers[moment])

    # Get epochs
    epochs = mne.Epochs(raw, events, event_id,
                        tmin = tmin, tmax = tmax, 
                        detrend = 1, preload = True)
    
    # Drop trials
    if drop:
        epochs = bad_epochs(epochs, picks="eeg", metric='var', outlier_side=1, significance_level=0.05)
        epoch_fname = dirs['epoch'] + '/epoch_clean_' + moment + '_s' + str(s) + '.fif'

    else:
        epoch_fname = dirs['epoch'] + '/epoch_' + moment + '_s' + str(s) + '.fif'

    # Save
    epochs.save(epoch_fname, overwrite = True)

    return epochs

In [ ]:
s = 1

epochs = get_epochs(s, 'enc1', event_id, -1, 4, drop = True)